Proviamo a scrivere con i tensori l'espressione di quello che accade nei neuroni. (tralasciamo la non linearità):
le prossime linee di codice rappresenterannno una rete neurale molto semplice, talmente semplice da non essere una vera rete neurale. 
In realtà è solo il primo strato. 

Prenderemo un vettore di input con 5 features.
Un vettore di output con tre features.
Un vettore bias, che si aggiunge a tutti i neuroni di output. 

qui aggiungici l'immagine di quello che sta accadentdo.

In [26]:
import torch 
import numpy as np 

x = torch.ones(size=(5,))
y = torch.ones(size=(3,))
w = torch.randn(size=(5,3), requires_grad=True)
b = torch.randn(size=(3,), requires_grad=True) #Il bias è tre perchè i neuroni di output è tre 

print(x)
print(x.shape)
print(y.shape)
print(w.shape)
print(x.T.shape)

tensor([1., 1., 1., 1., 1.])
torch.Size([5])
torch.Size([3])
torch.Size([5, 3])
torch.Size([5])


Come abbiamo già visto l'operatore matmul effettua il dot product, ovvero il prodotto tra vettori/matrici.
In questo momento sto moltiplicando un vettore 1x5 (vettore x), con la matrice 5x3 (matrice w).
Il risultato è un vettore 1x3, esattamente come dovrebbe essere, ovvero ha le stesse dimensioni di y.


In [27]:
#Implementiamo la moltiplicazione: l'operatore matmul, effettua un prodotto matrice vettore.
z = torch.matmul(x, w) + b
print(z)
print(z.shape)

tensor([ 0.6488, -0.7102, -2.6299], grad_fn=<AddBackward0>)
torch.Size([3])


In [28]:
#Calcolo di una loss: nn.functional è una sottolibreria di torch con dentro tutte le funzioni che vengono usate come loss.
loss = torch.nn.functional.binary_cross_entropy_with_logits(z , y) #with logits, si aspetta che io non abbia utilizzato la sigmoide di attivaxionie, e la fa lui. 


Quello che dovrei fare a questo punto, e che viene fatto in generale durante qualsiasia addestramento di un modello, è l'implementazione del gradiente e quindi della backpropagation. 

In realtà l'implementazione del gradiente non deve essere fatta manualmente per forza, ma viene fatto automaticamente dalla libreria torch. 

Il framework fa la backpropagation, con il tool authomatic differentiation graph, che gli permette di costruirsi un grafo di computazione che tiene traccia delle variabili e delle operazioni fatte.



In [ ]:
#Posso far vedere che un attributo dei tensori è proprio il gradiente: 
print("Gradient function:" , z.grad_fn)
print("Gradient:" , z.grad) #E' ancora non calcolato, lo farà .backward()

Gradient function: <AddBackward0 object at 0x7fdaba04aca0>


<AddBackward0 object at 0x7fdaba04c1c0> mi sta dicendo: guarda che questa cosa è una somma in una certa zona di memmoria.
E' effettivamente così, infatti z è un matmul() + b.
Il processo continua all'indietro in maniera autonoma. 

"AddBackward0" serve nel momento in cui gli chiedo di calcolare i gradienti, in questo caso lui si dovrà ricordare di calcolare la derivata della funzione add. Così ricostruirà il gradiente. 

Come faccio a chiedere di calcolare il gradiente e di applicarlo? con la funzione backward().

Una volta chiamata, fa una propagazione all'indietro di tutto il gradiente, fino alla radice del grafo computazionale,e salva i gradienti nell'attributo .grad di ogni tensore. 

Attenzione: lo fa solamente per i tensori per cui ho impostato requires_grad=True.
Sono solo i tensori in cui richiedo che i parametri vengano aggiornati, per esempio il tensore di input è logico che non abbia requires_grad=True.

In [30]:
loss.backward()

In [ ]:
#Posso printarmi i gradienti per esempio di w e b: 
print(w.grad)
print("\n",b.grad)

#NB:ancora non abbiamo aggiornato i parametri, abbiamo solamente calcolato il gradiente rispetto a w e b , e ce li siamo salvati nelle variabili w e b come loro attrivìbuti.
#Non sono stati cambiati i valori, l'ottimizzatore che anvcora non abbiamo introdotto ha come compito di aggiornare i parametri.
#L'ottimizzatore decide come usare i gradienti per aggiornare i parametri. (sgd, adam, rmsprop, etc...)

tensor([[-0.1144, -0.2235, -0.3109],
        [-0.1144, -0.2235, -0.3109],
        [-0.1144, -0.2235, -0.3109],
        [-0.1144, -0.2235, -0.3109],
        [-0.1144, -0.2235, -0.3109]])

 tensor([-0.1144, -0.2235, -0.3109])


Piccola appendice su come forzare la non richiesta del gradiente, sarà utile quando faremo fine tuning.

z1 = torch.matmul(x, w) + b 
print(z1.requires_grad)

---> True   questo perchè z combinazione di elementi che richiedono gradiente.


with torch.no_grad():
    z2 = torch.matmul(x, w) + b
    print(z2.requires_grad)

---> False

E' molot utile, per motivi di efficienza.
Infatti durante l'addestramento noi richiediamo il gradiente, ma quando il modello va in fase di evaluation (magari alla fine di un'epoca vogliamo provare il modello sul validation swt), nonserve più la necessità di richiedere il gradiente.

